In [1]:
from scipy.sparse import diags #linalg
from scipy.linalg import solve
import numpy as np
from numpy.linalg import inv, det
from matplotlib import pyplot as plt
from scipy.integrate import quad
from matplotlib import animation as animation
from IPython.display import HTML
from scipy.optimize import least_squares

In [7]:
def solver(T,dt, L2, Z, kd, ka):
    N = int(round(T/dt))
    t = np.linspace(0,T,N+1) #time discrete
    M = int(round(L/dx))
    x = np.linspace(0,L,M+1) #space discrete
    E = np.zeros((2*(M+1),N+1))
    O1 = np.zeros((M+1,N+1))
    O2 = np.zeros((M+1,N+1))
    O12 = np.zeros((2*M+2,N+1))
    O1lag = np.zeros((M+1,N+1))
    O2lag = np.zeros((M+1,N+1))
    O12lag = np.zeros((2*M+2,N+1))
    RHS12 = np.zeros((2*M+2))
    RHS3 = np.zeros((2*(M+1)))
    O1I = np.zeros((M+1,N+1))
    O2I = np.zeros((M+1,N+1))
    #X1 = np.zeros((2*(M+1),N+1))
    B1 = np.zeros((M+1))
    B2 = np.zeros((M+1))
    U1 = np.zeros((2*M+2)**2).reshape(2*M+2, 2*M+2)+np.diag(np.ones(2*M+2), 0)
    
    #initial conditions
    O1[0,:] = 7.63
    O2[0,:] = 1
    
    #special time step at n=0
    #Lagrangian concentration
    O1lag[0:1, 1] = O1[0,0]
    O1lag[1:M+1,1] = O1[1:M+1,0]+(v*dt-k*dx)*((O1[0:M,0]-O1[1:M+1,0])/dx)
    O2lag[0:1, 1] = O2[0,0]
    O2lag[1:M+1,1] = O2[1:M+1,0]+(v*dt-k*dx)*((O2[0:M,0]-O2[1:M+1,0])/dx)
    #Solving for D
    Z1 = np.sum(Z, axis = 1)
    L2 = np.divide(L2, Z1)
    L2 = L2.transpose() 
    #boundary conditions and constants
    B1[120] = 6.49 / Z1[120]
    B2[120] = 1 / Z1[120]
    B12 = np.concatenate((B1,B2))
    X = np.zeros((M+1)**2).reshape(M+1,M+1)
    X2 = np.zeros((M+1))
    X3 = ka*(1.024)**(T1-20)*O3*np.ones((M+1))  
    X4 = np.concatenate((X3,X2))
    #matrix for the n+1/2 time step
    Y11 = np.block([[(1/dt)*U,X],[X,(1/dt)*U]])
    Y22 = Y11
    Y33 = np.block([[-(ka*(1.024)**(T1-20))*U,-kd*U],[X,-kd*U]])
    O12lag[:,1] = np.concatenate((O1lag[:,1],O2lag[:,1]))
    #solution of the matrix
    M2 = U1/dt
    M1 = U1 / dt + np.block([[L2,X],[X,L2]])
    RHS12[:] = M2.dot(O12lag[:,1]) + B12 + Y33.dot(O12lag[:,1]) + X4
    O12[:,1] = np.linalg.solve(M1, RHS12)   #solution of the FEM
    O12[0,1] = 7.63
    O12[M+1,1] = 1
    U1 = diags([1], [0,], shape=(M+1, M+1)).toarray()
    #matrix for the n+1 time step
    Y1 = np.block([[(rho/dt+A*ka*(1.024)**(T1-20))*U,A*kd*U],[X,(rho/dt+A*kd)*U]])
    Y2 = np.block([[A*ka*(1.024)**(T1-20)*U,A*kd*U],[X,A*kd*U]])
    Y3 = np.block([[(rho/dt)*U,X],[X,(rho/dt)*U]])
    RHS3[:] = Y2.dot(O12lag[:,1]) + Y3.dot(O12[:,1])
    E[:,1] = np.linalg.solve(Y1, RHS3)
    E[0,1] = 7.63
    O1[:,1] = E[0:M+1,1]
    O2[:,1] = E[M+1:2*(M+1),1]
    
    for n in range(1,N):
        #update all matrices
        #Lagrangian concentration
        U1 = np.zeros((2*M+2)**2).reshape(2*M+2, 2*M+2)+np.diag(np.ones(2*M+2), 0)
        O1lag[0:1, n+1] = O1[0,0]
        O1lag[1:M+1,n+1] = O1[1:M+1,n]+(v*dt-k*dx)*((O1[0:M,n]-O1[1:M+1,n])/dx) - kd*O2[1:M+1,n]+ka*(1.024)**(T1-20)*(O3-O1[1:M+1,n])
        O2lag[0:1, n+1] = O2[0,0]
        O2lag[1:M+1,n+1] = O2[1:M+1,n]+(v*dt-k*dx)*((O2[0:M,n]-O2[1:M+1,n])/dx) - kd*O2[1:M+1,n]
        #Solving for D
        Z1 = np.sum(Z, axis = 1)
        L2 = np.divide(L2, Z1)
        L2 = L2.transpose() 
        #boundary conditions and constants
        B1[120] = 6.49 / Z1[120]
        B2[120] = 1 / Z1[120]
        B12 = np.concatenate((B1,B2))
        X = np.zeros((M+1)**2).reshape(M+1,M+1)
        X2 = np.zeros((M+1))
        X3 = ka*(1.024)**(T1-20)*O3*np.ones((M+1))  
        X4 = np.concatenate((X3,X2))
        #matrix for the n+1/2 time step
        Y11 = np.block([[(1/dt)*U,X],[X,(1/dt)*U]])
        Y22 = Y11
        Y33 = np.block([[-(ka*(1.024)**(T1-20))*U,-kd*U],[X,-kd*U]])
        O12lag[:,n+1] = np.concatenate((O1lag[:,n+1],O2lag[:,n+1]))
        #solution of the matrix
        M2 = U1/dt
        M1 = U1 / dt + + np.block([[L2,X],[X,L2]])
        RHS12[:] = M2.dot(O12lag[:,n+1]) + B12 + Y33.dot(O12lag[:,n+1]) + X4
        O12[:,n+1] = np.linalg.solve(M1, RHS12)   #solution of the FEM
        O12[0,n+1] = 7.63
        O12[M+1,n+1] = 1
        
        #matrix for the n+1 time step
        Y1 = np.block([[(rho/dt+A*ka*(1.024)**(T1-20))*U,A*kd*U],[X,(rho/dt+A*kd)*U]])
        Y2 = np.block([[A*ka*(1.024)**(T1-20)*U,A*kd*U],[X,A*kd*U]])
        Y3 = np.block([[(rho/dt)*U,X],[X,(rho/dt)*U]])
        RHS3[:] = Y2.dot(O12lag[:,n+1]) + Y3.dot(O12[:,n+1])
        E[:,n+1] = np.linalg.solve(Y1, RHS3)
        O1[1:M+1,n+1] = E[1:M+1,n+1]
        O2[:,n+1] = E[M+1:2*(M+1),n+1]
            
    return   O1,O2, x, t

In [8]:
#Solving for matrices L1, L2, L3
T =  2000
L = 1770
P = 7
hb = 1
rho = 1       #density of DO
l = 10          #element distance
K = 1000         #dispersion coefficient K=diffusion+dispersitivity*fv
A = 20
A1 = 1
v = 1 # 1.38888888889
k = 0
T1 = 22.18
#kd = 0.00000115740740741
#kr = 0.00000578703703704
#ka = kr/(1.024**(T1-20))
O3 = 14.61996-0.4042*T1+0.00842*T1**2-0.00009*T1**3
dt = 10        #time step size
dx = 10
N = int(round(T/dt))          #no. of time steps
M = int(round(L/dx))          #no. of nodes

#Solving for A2
def integrand(x):
    return (-1/l)*K*A*(-1/l)
ans12, err = quad(integrand, 0, l)
def integrand(x):
    return (-1/l)*K*A*(1/l)
ans13, err = quad(integrand, 0, l)
def integrand(x):
    return (1/l)*K*A*(-1/l)
ans14, err = quad(integrand, 0, l)
def integrand(x):
    return (1/l)*K*A*(1/l)
ans15, err = quad(integrand, 0, l)
L2 = diags([ans14,(ans12+ans15),ans13], [-1,0,1], shape=(M+1, M+1)).toarray()
L2[0,0] = ans12
L2[M,M] = ans15

#Solving for M = A1
def integrand(x):
    return (1-x/l)*A*(1-x/l)
ans20, err = quad(integrand, 0, l)
def integrand(x):
    return (1-x/l)*A*(x/l)
ans21, err = quad(integrand, 0, l)
def integrand(x):
    return (x/l)*A*(1-x/l)
ans22, err = quad(integrand, 0, l)
def integrand(x):
    return (x/l)*A*(x/l)
ans23, err = quad(integrand, 0, l)
Z = diags([ans22,(ans20+ans23),ans21], [-1,0,1], shape=(M+1, M+1)).toarray()
Z[0, 0] = ans20
Z[M, M] = ans23
#unit matrix
U = np.zeros((M+1)**2).reshape(M+1, M+1)+np.diag(np.ones(M+1), 0)

#(O1,O2, x, t, O1lag, O1I, O2I) = solver(T,dt, L2, Z)